In [ ]:
import re
import sqlite3
from datetime import datetime

conn = sqlite3.connect("triage.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS triage_responses (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id TEXT,
    triage_level TEXT,
    reasoning TEXT,
    urgent_evaluation_needed TEXT,
    patient_actions TEXT,
    clinician_tasks TEXT,
    disclaimer TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

# --- Raw LLM output ---
llm_text = """
- **Id** : 67891
- **Triage Level**: Emergency
- **Reasoning**: The patient has a history of hypertension and diabetes, and now presents with sudden chest pain and shortness of breath. These are high‑risk symptoms for acute cardiac events, requiring immediate attention.
- **Urgent Evaluation Needed**: Yes; ECG, cardiac enzymes, and continuous monitoring are necessary.
- **Patient Actions**: Seek emergency medical care immediately.
- **Clinician Tasks**: Perform ECG, order cardiac enzyme tests, monitor vital signs, and prepare for possible advanced cardiac intervention.
- **Disclaimer**: This information is based on the provided context and past records. It is not a substitute for professional medical advice. Patients should consult a licensed healthcare provider for diagnosis and treatment.
"""

# --- Parse function ---
def parse_llm_output(text: str) -> dict:
    fields = {}
    for line in text.splitlines():
        match = re.match(r"-\s*\*\*(.+?)\*\*\s*:? (.*)", line.strip())
        if match:
            key, value = match.groups()
            key = key.lower().replace(" ", "_")
            fields[key] = value.strip()
    return fields

parsed = parse_llm_output(llm_text)

# --- Insert or Update ---
def save_response(response: dict, update_latest=False):
    if update_latest:
        cursor.execute("""
        UPDATE triage_responses
        SET triage_level=?, reasoning=?, urgent_evaluation_needed=?, 
            patient_actions=?, clinician_tasks=?, disclaimer=?, created_at=?
        WHERE user_id=?
        """, (
            response.get("triage_level"),
            response.get("reasoning"),
            response.get("urgent_evaluation_needed"),
            response.get("patient_actions"),
            response.get("clinician_tasks"),
            response.get("disclaimer"),
            datetime.now(),
            response.get("id")
        ))
    else:
        cursor.execute("""
        INSERT INTO triage_responses 
        (user_id, triage_level, reasoning, urgent_evaluation_needed, patient_actions, clinician_tasks, disclaimer)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (
            response.get("id"),
            response.get("triage_level"),
            response.get("reasoning"),
            response.get("urgent_evaluation_needed"),
            response.get("patient_actions"),
            response.get("clinician_tasks"),
            response.get("disclaimer")
        ))
    conn.commit()

# --- Save parsed response ---
save_response(parsed, update_latest=False)  # append mode

# Close connection
conn.close()

In [ ]:
import sqlite3

conn = sqlite3.connect("triage.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM triage_responses")
rows = cursor.fetchall()


for row in rows:
    print(row)

conn.close()

In [ ]:
conn = sqlite3.connect("triage.db")
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

cursor.execute("SELECT * FROM triage_responses WHERE user_id = ?", ("12345",))
row = cursor.fetchone()
conn.close()

print(row)
print("User ID:", row["user_id"])
print("Triage Level:", row["triage_level"])
print("Reasoning:", row["reasoning"])
print("Patient Actions:", row["patient_actions"])

In [ ]:
import sqlite3

conn = sqlite3.connect("triage.db")
cursor = conn.cursor()

# Fetch the complete record for a given user_id
cursor.execute("SELECT * FROM triage_responses WHERE user_id = ?", ("12345",))
row = cursor.fetchone()

print(row)   # prints the full record as a tuple
conn.close()

In [ ]:
{
  "kind": "doctor",
  "doctor_name": "Raskesh",
  "date": "2025-11-30",
  "time": "09:00",
  "preferred_start": "2025-11-30 09:00"
}
{
  "kind": "service",
  "service": "wellness check",
  "date": "2025-11-30",
  "time": "09:00",
  "preferred_start": "2025-11-30 09:00"
}{
  "kind": "lab",
  "test": "blood test",
  "location": "Indiranagar",
  "date": "2025-11-30",
  "time": "09:00",
  "preferred_start": "2025-11-30 09:00"
}
{
  "kind": "disease",
  "disease": "diabetes",
  "specialty": "endocrinologist",
  "test": "HbA1c",
  "date": "2025-11-30",
  "time": "09:00",
  "preferred_start": "2025-11-30 09:00"
}